In [25]:
''' Dependencies '''
import pandas as pd
import numpy as np
import scrapy
from datetime import date, datetime, time, timedelta

In [3]:
# csv_table = pd.read_table("../data_samples/middleriver-00480.tsv", sep="\t", skiprows=26)

In [4]:
# csv_table.to_csv(f'../data_samples/middleriver-00480.csv', index=False)

In [5]:
mr_df = pd.read_csv('../data_samples/middleriver-00480.csv')
mr_df.head()

,agency_cd,site_no,datetime,tz_cd,257812_00480,257812_00480_cd
0,5s,15s,20d,6s,14n,10s
1,USGS,11312676,2023-02-16 00:00,PST,0.2,P
2,USGS,11312676,2023-02-16 00:15,PST,0.2,P
3,USGS,11312676,2023-02-16 00:30,PST,0.2,P
4,USGS,11312676,2023-02-16 00:45,PST,0.2,P


In [6]:
url_init = "https://waterservices.usgs.gov/nwis/iv/?sites="
url_end = "&siteStatus=all&format=rdb"
monitor_loc = "11455385"
param_codes = ['99133', '00400', '00480']  # nitrates, pH, salinity
end_date = date.today()
past_week = end_date - timedelta(days=7)
start_date = end_date - timedelta(days=1)


if past_week == end_date:
    urls = ['{}{}&parameterCd={}&startDT={}&endDT={}{}'.format(url_init, monitor_loc, param_codes[0], past_week, end_date, url_end),
            '{}{}&parameterCd={}&startDT={}&endDT={}{}'.format(url_init, monitor_loc, param_codes[1], past_week, end_date, url_end),
            '{}{}&parameterCd={}&startDT={}&endDT={}{}'.format(url_init, monitor_loc, param_codes[2], past_week, end_date, url_end)]
else:
    urls = ['{}{}&parameterCd={}&startDT={}&endDT={}{}'.format(url_init, monitor_loc, param_codes[0], start_date, end_date, url_end),
            '{}{}&parameterCd={}&startDT={}&endDT={}{}'.format(url_init, monitor_loc, param_codes[1], start_date, end_date, url_end),
            '{}{}&parameterCd={}&startDT={}&endDT={}{}'.format(url_init, monitor_loc, param_codes[2], start_date, end_date, url_end)]
print(urls)

['https://waterservices.usgs.gov/nwis/iv/?sites=11455385&parameterCd=99133&startDT=2023-02-25&endDT=2023-02-26&siteStatus=all&format=rdb', 'https://waterservices.usgs.gov/nwis/iv/?sites=11455385&parameterCd=00400&startDT=2023-02-25&endDT=2023-02-26&siteStatus=all&format=rdb', 'https://waterservices.usgs.gov/nwis/iv/?sites=11455385&parameterCd=00480&startDT=2023-02-25&endDT=2023-02-26&siteStatus=all&format=rdb']


In [7]:
urls = ['{}{}&parameterCd={}&startDT={}&endDT={}{}'.format(url_init, monitor_loc, code, past_week, end_date, url_end) for code in param_codes]
urls

['https://waterservices.usgs.gov/nwis/iv/?sites=11455385&parameterCd=99133&startDT=2023-02-19&endDT=2023-02-26&siteStatus=all&format=rdb',
 'https://waterservices.usgs.gov/nwis/iv/?sites=11455385&parameterCd=00400&startDT=2023-02-19&endDT=2023-02-26&siteStatus=all&format=rdb',
 'https://waterservices.usgs.gov/nwis/iv/?sites=11455385&parameterCd=00480&startDT=2023-02-19&endDT=2023-02-26&siteStatus=all&format=rdb']

In [24]:
date = date.today()
# past_week = end_date - timedelta(days=7)
start_date = date - timedelta(days=1)

start_dt_iso = datetime.combine(start_date, time.min).isoformat()
end_dt_iso = datetime.combine(start_date, time.max).isoformat()

print(start_dt_iso, end_dt_iso)

2023-02-25T00:00:00 2023-02-25T23:59:59.999999


In [26]:
# Short format date
short_format = start_date.strftime("%Y%m%d")
short_format

'20230225'

In [40]:
import boto3
import io
from io import BytesIO

In [41]:
''' DELETE AFTER USE!!! '''


In [42]:
df = pd.read_csv(r'../data_samples/ryerisland-00400.csv')
df.head()

,agency_cd,site_no,datetime,tz_cd,234489_00400,234489_00400_cd
0,5s,15s,20d,6s,14n,10s
1,USGS,11455385,2023-02-24 00:00,PST,7.9,P
2,USGS,11455385,2023-02-24 00:15,PST,7.9,P
3,USGS,11455385,2023-02-24 00:30,PST,7.9,P
4,USGS,11455385,2023-02-24 00:45,PST,8.0,P


In [43]:
bucket_name = 'water-pipeline'
key_prefix = 'ryerisland/'

In [52]:
# Write to s3
s3 = boto3.client('s3', 
                aws_access_key_id = AWS_ACCESS_KEY_ID,
                aws_secret_access_key = AWS_SECRET_ACCESS_KEY)
# buffer = BytesIO()
# csv_bytes = df.getvalue().encode("utf-8")
filename = key_prefix + 'ryerisland-00480.csv'
csv_buffer = df.to_csv(index=False)
s3.put_object(Bucket=bucket_name, Key=filename, Body=csv_buffer)

{'ResponseMetadata': {'RequestId': 'R8MCRRES1CS6X6WR',
  'HostId': '6cRtvErrmadCt3C0U5KRlnC2EUunVZyLukwyGiIl82TBhwu6DUccNnZIkT9ILrQlkun1NHICQKsNOpjJf1g/9g==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '6cRtvErrmadCt3C0U5KRlnC2EUunVZyLukwyGiIl82TBhwu6DUccNnZIkT9ILrQlkun1NHICQKsNOpjJf1g/9g==',
   'x-amz-request-id': 'R8MCRRES1CS6X6WR',
   'date': 'Mon, 27 Feb 2023 00:30:34 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3efdf7d17c7a11184ab9b968c466f483"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"3efdf7d17c7a11184ab9b968c466f483"',
 'ServerSideEncryption': 'AES256'}

In [51]:
from config.config import *

ModuleNotFoundError: No module named 'config.config'; 'config' is not a package